In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

client = genai.Client()

prompt = (
    "Create a detailed photorealistic strawberry leaf texture.",
    "No stems and the background should be blue (far from any color in the leaf) for easy removal.",
    "It should be flat and have minimal shadows.(Like a texture map.)",
    "Dont have highlights or shiny parts.",
    "Only create the terminal/central/apical leaflet of the trifoliate leaf.",

)

response = client.models.generate_content(
    model="gemini-2.5-flash-image-preview",
    contents=[prompt],
)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        central = Image.open(BytesIO(part.inline_data.data))
        central.save("generated_image.png")
        display(central)  # Display inline in notebook

In [ ]:
prompt = (
    "This is the texture of the terminal/central/apical leaflet of the trifoliate strawberry leaf.",
    "Please create the bump map for this texture.",
    "A bump map is a grayscale image that represents surface height variations.",
)

response = client.models.generate_content(
    model="gemini-2.5-flash-image-preview",
    contents=[prompt, central],
)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    elif part.inline_data is not None:
        bump = Image.open(BytesIO(part.inline_data.data))
        bump.save("generated_image.png")
        display(bump)  # Display inline in notebook

In [ ]:
# Get the background color (first pixel)
clip_color = central.getpixel((0, 0))
print("Background color:", clip_color)

# Function to convert clip color to alpha
def color_to_alpha(image, clip_color, tolerance=30):
    """
    Convert a specific color to transparent alpha channel
    
    Args:
        image: PIL Image in RGB or RGBA mode
        clip_color: RGB tuple of color to make transparent
        tolerance: How close colors need to be to clip_color (0-255)
    """
    # Convert to RGBA if not already
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    
    # Get image data as array
    data = image.getdata()
    new_data = []
    
    for pixel in data:
        r, g, b = pixel[:3]  # Get RGB values
        cr, cg, cb = clip_color
        
        # Calculate color distance
        distance = ((r - cr) ** 2 + (g - cg) ** 2 + (b - cb) ** 2) ** 0.5
        
        if distance <= tolerance:
            # Make transparent
            new_data.append((r, g, b, 0))
        else:
            # Keep original with full alpha
            new_data.append((r, g, b, 255))
    
    # Create new image with alpha
    result = Image.new('RGBA', image.size)
    result.putdata(new_data)
    return result

# Apply color clipping
central_clipped = color_to_alpha(central, clip_color, tolerance=50)
central_clipped.save("central_clipped.png")
display(central_clipped)
central_bump_clipped = color_to_alpha(bump, clip_color, tolerance=50)
central_bump_clipped.save("central_bump_clipped.png")
display(central_bump_clipped)



In [ ]:
def get_max_bounding_box_and_crop(images):
    """
    Find the maximum bounding box that includes all non-transparent pixels
    from multiple PIL images and crop all images to that bounding box.
    
    Args:
        images: List of PIL Images (should have alpha channel for transparency)
    
    Returns:
        List of cropped PIL Images, all with the same size
    """
    if not images:
        return []
    
    # Initialize bounding box coordinates
    min_x, min_y = float('inf'), float('inf')
    max_x, max_y = 0, 0
    
    # Find the maximum bounding box across all images
    for img in images:
        # Convert to RGBA if not already
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        
        # Get the bounding box of non-transparent pixels
        bbox = img.getbbox()
        
        if bbox:  # If image has non-transparent pixels
            left, top, right, bottom = bbox
            min_x = min(min_x, left)
            min_y = min(min_y, top)
            max_x = max(max_x, right)
            max_y = max(max_y, bottom)
    
    # If no non-transparent pixels found in any image
    if min_x == float('inf'):
        return images  # Return original images
    
    # Calculate the maximum bounding box
    max_bbox = (int(min_x), int(min_y), int(max_x), int(max_y))
    print(f"Maximum bounding box: {max_bbox}")
    print(f"Crop size: {max_x - min_x} x {max_y - min_y}")
    
    # Crop all images to the maximum bounding box
    cropped_images = []
    for img in images:
        # Convert to RGBA if not already
        if img.mode != 'RGBA':
            img = img.convert('RGBA')
        
        # Crop to the maximum bounding box
        cropped = img.crop(max_bbox)
        cropped_images.append(cropped)
    
    return cropped_images

# Example usage with your images
images_to_crop = [central_clipped, central_bump_clipped]
cropped_images = get_max_bounding_box_and_crop(images_to_crop)

# Save and display the cropped images
if cropped_images:
    central_cropped = cropped_images[0]
    bump_cropped = cropped_images[1]
    
    central_cropped.save("central_cropped.png")
    bump_cropped.save("bump_cropped.png")
    
    print("Central leaf cropped:")
    display(central_cropped)
    
    print("Bump map cropped:")
    display(bump_cropped)

In [ ]:
import numpy as np

def bump_to_normal(bump_image, strength=1.0):
    """
    Convert a grayscale bump map to a normal map.
    
    Args:
        bump_image: PIL Image in grayscale or RGB (will use luminance)
        strength: Multiplier for the normal strength (default 1.0)
    
    Returns:
        PIL Image: Normal map in RGB format
    """
    # Convert to grayscale array if needed
    if bump_image.mode == 'RGBA':
        # Use RGB channels, ignore alpha
        bump_array = np.array(bump_image.convert('RGB'))
        bump_gray = np.dot(bump_array[...,:3], [0.299, 0.587, 0.114])
    elif bump_image.mode == 'RGB':
        bump_array = np.array(bump_image)
        bump_gray = np.dot(bump_array, [0.299, 0.587, 0.114])
    else:
        bump_gray = np.array(bump_image.convert('L'))
    
    # Normalize to 0-1 range
    bump_gray = bump_gray.astype(np.float32) / 255.0
    
    # Calculate gradients (surface derivatives)
    grad_x = np.zeros_like(bump_gray)
    grad_y = np.zeros_like(bump_gray)
    
    # Calculate X gradient (horizontal)
    grad_x[:, 1:] = bump_gray[:, 1:] - bump_gray[:, :-1]
    grad_x[:, 0] = grad_x[:, 1]  # Duplicate edge
    
    # Calculate Y gradient (vertical) 
    grad_y[1:, :] = bump_gray[1:, :] - bump_gray[:-1, :]
    grad_y[0, :] = grad_y[1, :]  # Duplicate edge
    
    # Apply strength multiplier
    grad_x *= strength
    grad_y *= strength
    
    # Create normal vectors
    # Normal = (-dx, -dy, 1) normalized
    normal_x = -grad_x
    normal_y = -grad_y  
    normal_z = np.ones_like(grad_x)
    
    # Normalize the normal vectors
    length = np.sqrt(normal_x**2 + normal_y**2 + normal_z**2)
    normal_x /= length
    normal_y /= length  
    normal_z /= length
    
    # Convert from [-1,1] to [0,255] range
    # Normal map uses: R=X, G=Y, B=Z
    normal_r = ((normal_x + 1.0) * 0.5 * 255).astype(np.uint8)
    normal_g = ((normal_y + 1.0) * 0.5 * 255).astype(np.uint8)  
    normal_b = ((normal_z + 1.0) * 0.5 * 255).astype(np.uint8)
    
    # Stack into RGB image
    normal_rgb = np.stack([normal_r, normal_g, normal_b], axis=-1)
    
    # Convert back to PIL Image
    normal_map = Image.fromarray(normal_rgb, 'RGB')
    
    return normal_map

# Generate normal map from your bump map
normal_map = bump_to_normal(bump_cropped, strength=2.0)
normal_map.save("normal_map.png")

print("Normal map generated:")
display(normal_map)